# Sistemas de Recomendación - Netflix Prize Challenge

En este notebook vamos a implementar un sistema de recomendación a través de un filtro colaborativo.

Algunas referencias útiles, además de las mencionadas en la presentación:
* https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system
* https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101

El dataset puede ser descargado [acá](https://www.kaggle.com/netflix-inc/netflix-prize-data)

Importante leer el archivo `README.md` para la descripción de los archivos.

## **Big Data**


Uno de los desafíos que plantea este dataset es que es bastante "grande". Esto quiere decir que, si lo cargamos completo, ocupa bastante lugar en nuestra memoria RAM. Además, cada tarea puede llevar mucho tiempo. Entonces, es necesario plantear una estrategia para abordarlo. Existen varias posibilidades, mencionamos algunas:
1. Recortar una parte del dataset con la que sí podamos trabajar. Esta parte tiene que ser lo suficientemente representativa del set original. Para estar seguros de ello es fundamental hacer una buena exploración de datos. Con ese recorte, entrenamos y evaluamos nuestro modelo, y optimizamos parámetros (CV). Una vez que ya estamos seguro de que nuestro flujo de trabajo es apropiado, podemos probar agrandar la porción de datos con la que entrenamos o utilizar otro recorte del dataset. Eventualmente, podemos llegar a usar todo el dataset para entrenar y evaluar si nuestra computadora lo permite. **NOTA**: el recorte se hace para que se pueda cargar en memoria los datos pero también para que cada iteración lleve un tiempo razonable.
2. **Aprendizaje incremental**: algunos modelos puede ser entrenados mostrándoles el dataset de a pedazos. Es decir, no necesitan ver todo el dataset a la vez. Un ejemplo son las redes neuronales, que "ven" muchas pasadas del dataset en *epochs* y *minibatches*. Algunos modelos en Scikit-learn tienen la función `partial_fit` que permite hacer eso. Pueden leer un poco al respecto [acá](https://scikit-learn.org/stable/modules/computing.html). Pandas también tiene funciones que permiten cargar el dataset de a trozos.
3. Utilizar servicios en la nube. Esta opción no es excluyente con las anteriores. Antes de utilizar algún entorno en la nube, está bueno haber hecho pruebas en nuestra computadora y ya haber optimizado bastante el flujo de trabajo. Recuerden que los servicios en la nube se pagan.
4. Existen librerías orientadas a trabajar con grandes datos. Un ejemplo es [Dask](https://dask.org/).

## 0. Algunos preliminares

Mientras miran el estado de la memoria RAM, crear un arreglo 2-D de unos en numpy de forma (10000,10000).

In [ ]:
import numpy as np

In [ ]:
N = 10000
unos = COMPLETAR

In [ ]:
unos

In [ ]:
del unos # Borra la variable

¿Qué tipo de dato es `ones`?¿Y sus elementos? Crear el mismo arreglo, pero convertir los elementos en *np.int8*

In [ ]:
unos = COMPLETAR.astype(COMPLETAR)

In [ ]:
unos

Por las dudas reiniciar el Kernel antes de continuar y correr a partir de la sección siguiente.

## 1. Carga de Datos y preparación del Dataset

Vamos a empezar cargando uno de los archivos con calificaciones para explorarlo. Como son archivos grandes y van a ocupar bastante lugar en memoria, no vamos a cargar la última columna con fechas.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector

In [ ]:
def load_data(name):
    df = pd.read_csv(name, header = None, names = ['User','Rating'], usecols = [0,1])
    
    # A veces forzar un tipo de dato hace que se ahorre mucho lugar en memoria.
    df['Rating'] = df['Rating'].astype(float) 
    return df


df1 = load_data('Datasets/netflix-prize-data/combined_data_1.txt')
print(df1.shape)

In [ ]:
df1.head()

¿Cómo sabemos a qué película corresponde cada calificación? Contar cuántas películas hay en `df1` e identificarlas. Para ello, cargamos `movie_titles.csv`. Como no nos interesa el año, no lo traemos.

In [ ]:
df_title = pd.read_csv('Datasets/netflix-prize-data/movie_titles.csv', encoding = "ISO-8859-1",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])
df_title.head()

De esta forma, podemos obtener el nombre de una película dado su Id

In [ ]:
movie_id = 1
print(df_title.loc[movie_id].Name)

Para contar cuántos identificadores hay, vamos a usar la siguiente información: al lado del identificador de la película, la columna `Rating` de `df1` tiene un `NaN`.

In [ ]:
movies_ids_df1 = df1.User[df1.Rating.isna()].values
print(movies_ids_df1)
print(len(movies_ids_df1))

¿En qué formato está? Si queremos usarlo para pasar de identificador al nombre, debemos llevarlo a enteros. Asumimos que no hay ningun repetido:

In [ ]:
movies_ids_df1 = np.arange(1,len(movies_ids_df1) + 1)
print(movies_ids_df1)

### Movie Id

Intentaremos agregar una columna al Dataframe con el Id de la película a la que corresponde la calificación. Para ello, vamos a necesitar saber dónde están ubicados los identificadores.

Primero, seleccionamos los índices donde aparecen los movies_ids

In [ ]:
df1_nan = pd.DataFrame(pd.isnull(df1.Rating))
df1_nan = df1_nan[df1_nan['Rating'] == True]
idx_movies_ids = df1_nan.index.values
print(idx_movies_ids)

Queremos crear un vector de tantas instancias como df1, donde en cada lugar esté movie_id a cual corresponde la calificación. Como tenemos los índices donde está cada movie_id, podemos obtener cuántas calificaciones hay de cada película.

In [ ]:
# Agregamos el indice de la ultima instancia del dataframe
idx_movies_ids = np.append(idx_movies_ids,df1.shape[0])
cantidad_criticas = np.diff(idx_movies_ids)
cantidad_criticas

In [ ]:
columna_movie_id = np.array([])
for i in range(cantidad_criticas.size):
    aux = np.full(cantidad_criticas[i], movies_ids_df1[i])
    columna_movie_id = np.concatenate((columna_movie_id, aux))

Agregamos esa columna al dataset

In [ ]:
df1['movie_id'] = columna_movie_id
del columna_movie_id

df1.dropna(inplace = True)
df1['User'] = df1['User'].astype(int)
df1['movie_id'] = df1['movie_id'].astype(np.int16)
df1['Rating'] = df1['Rating'].astype(np.int8)

gc.collect()

In [ ]:
df1

Ya contamos con un dataframe con calificaciones de usuarios a películas.

Una opción es guardar el dataset modificado en nuevo archivo y, a partir de ahora, trabajar con esa versión. Esto hará que no tengamos que hacer el preprocesamiento cada vez que empecemos a trabajar y, además, ahorrarnos toda la "basura" que Python pueda ir dejando en la RAM.

**Ejercicio**: guardar el dataset modificado en un nuevo archivo.

In [ ]:
if False:
    pass
    #completar

## 2. Exploración del Dataset

Responder las siguientes preguntas, siempre que se pueda con un lindo gráfico (¡pensar bien cómo!):

1. ¿Cuántos usuarios únicos hay?
2. ¿Cuántas películas calificó cada usuario?
3. ¿Cómo es la distribución de las calificaciones?¿Pueden concluir algo de ese gráfico?
4. ¿Cuál es la película con más calificaciones?¿Cuántas tiene?¿Y la que menos calificaciones tiene?

In [ ]:
# COMPLETAR

### Opcional: filtrar películas con pocos ratings

In [ ]:
# COMPLETAR

## 3. Entrenamiento

Para entrenar el sistema de recomendación vamos a usar la biblioteca Surprise. Recomendamos tener abierta la [documentación](https://surprise.readthedocs.io/en/stable/getting_started.html) a medida que van a través de este notebook.

### 3.1 Dataset y Train/test split

Primero, llevamos el dataset al formato que le gusta a la biblioteca. ¿En qué orden tienen que estar los atributos?. Investigar qué hace la clase `Reader` y cuáles son sus parámetros.

In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

In [ ]:
reader = Reader()

Luego, creamos el `Dataset` de Surprise usando `Dataset.load_from_df`

In [ ]:
N_filas = 1000000 # Limitamos el dataset a N_filas

data = Dataset.load_from_df(df1[[COMPLETAR][:COMPLETAR], COMPLETAR)

¿Cómo les parece que es mejor hacer el split?¿Dejando películas en test, usuarios o combinaciones?

In [ ]:
trainset, testset = COMPLETAR

### 3.2 Entrenamiento

Vamos a entrenar un algoritmo SVD. Explorar sus parámetros y su funcionamiento.

In [ ]:
from surprise import SVD
algo = SVD()

Entrenamos sobre el `trainset`

In [ ]:
algo.COMPLETAR(COMPLETAR)

Y predecimos sobre el `testset`. Notar que para predecir sobre un conjunto de test se usa la función `test`.

In [ ]:
predictions = algo.COMPLETAR(COMPLETAR)

Explorar las característica de `predictions` y alguno de sus elementos

In [ ]:
predictions[1]

En cambio, si queremos predecir para un usuario y una película en particular, usamos la función `predict`

In [ ]:
algo.predict(237993,175)

Exploremos un usuario, veamos cuáles películas le gustaron y cuáles les recomienda el sistema.

Priomero, buscamos qué películas le gustaron

In [ ]:
usuario = 237993
rating = 5   # le pedimos peliculas a las que haya puesto 4 o 5 estrellas
df_user = df1[(df1['User'] == COMPLETAR) & (df1['Rating'] >= COMPLETAR)]
df_user = df_user.reset_index(drop=True)
df_user['Name'] = df_title['Name'].loc[df_user.movie_id].values
df_user

Creamos donde vamos a guardar las recomendaciones

In [ ]:
recomendaciones_usuario = df_title.iloc[:4499].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

Sacamos del dataframe todas las películas que ya sabemos que vio

In [ ]:
usuario_vistas = df1[COMPLETAR]
print(usuario_vistas.shape)

recomendaciones_usuario.drop(COMPLETAR, inplace = True)
recomendaciones_usuario = recomendaciones_usuario.reset_index()

Y hacemos las recomendaciones

In [ ]:
recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['Movie_Id'].apply(lambda x: algo.predict(usuario, x).est)

In [ ]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))

### 3.3 Evaluación

Para el conjunto de `testset`, evaluamos el error RMSE entre las predicciones y las verdaderas calificaciones que le habían dado a las películas. Para eso, buscar en la documentación cómo se hace.

In [ ]:
from surprise import COMPLETAR

COMPLETAR

La próxima clase continuaremos con Optimización de Parámetros y algunos comentarios más sobre cómo trabajar con datasets grandes.